## DoorDash delivery time Prediction


## Problem Statement :
## To predict the estimated time taken for a delivery

## Columns
1) **market_id**: city/region in which DoorDash operates e.g. 1->california  2-> new york
2) **created_at**: Timestamp when order created
3) **actual_delivery_time**: Timestamp when order delivered
4) **store_id**: the restaurant the order was submitted for store_primary_category
 e.g. market_id- 1, different outlets, each outlet have id for cuisine category
5) **store_primary_category**: cuisine category of the restaurant
6) **order_protocol**: Order receive in which mode, e.g. 1-> online, 2-> call
7) **total_items**: total number of items in the order
8) **subtotal**: total value of the order submitted (in cents)
9) **num_distinct_items**: number of distinct items included in the order
10) **min_item_price**: price of the item with the least cost in the order (in cents)
11) **max_item_price**: price of the item with the highest cost in the order (in cents)
12) **total_onshift_dashers**: Number of available dashers who are within 10 miles of the store at the time of order creation
13) **total_busy_dashers**: Subset of above total_onshift_dashers who are currently working on an order
14) **total_outstanding_orders**: Number of orders within 10 miles of this order that are currently being processed.
15) **estimated_order_place_duration**: Estimated time for the restaurant to receive the order from DoorDash (in seconds) i.e. to ready the order
16) **estimated_store_to_consumer_driving_duration**: Estimated travel time between store and consumer (in seconds)




## Step 1: Data Gathering

In [197]:
import pandas as pd
path = r"C:\Users\orbad\Downloads\Dataset-Regression.csv"
df = pd.read_csv(path)
df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,22:24:17,23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,21:49:25,22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,20:39:28,21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,21:21:45,22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,02:40:36,03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


## Calculate Total delivery time 
## Available Dashers
## Estimated Time Duration

In [198]:
df['total_deliveryTime'] = (
    pd.to_timedelta(df['actual_delivery_time']) -
    pd.to_timedelta(df['created_at'])
).dt.total_seconds()

df['total_available_dashers'] = (
    (df['total_onshift_dashers']) -
    (df['total_busy_dashers'])
)
df['estimated_time_duration'] = (
    (df['estimated_order_place_duration']) +
    (df['estimated_store_to_consumer_driving_duration'])
)

df.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,total_deliveryTime,total_available_dashers,estimated_time_duration
0,1.0,22:24:17,23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,3779.0,19.0,1307.0
1,2.0,21:49:25,22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,4024.0,-1.0,1136.0
2,3.0,20:39:28,21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,1781.0,1.0,1136.0
3,3.0,21:21:45,22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,3075.0,0.0,735.0
4,3.0,02:40:36,03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,2390.0,0.0,1096.0


## Data Quality Checks

In [199]:
df.shape

(197428, 19)

In [200]:
df.columns

Index(['market_id', 'created_at', 'actual_delivery_time', 'store_id',
       'store_primary_category', 'order_protocol', 'total_items', 'subtotal',
       'num_distinct_items', 'min_item_price', 'max_item_price',
       'total_onshift_dashers', 'total_busy_dashers',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration', 'total_deliveryTime',
       'total_available_dashers', 'estimated_time_duration'],
      dtype='str')

In [201]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 19 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  str    
 2   actual_delivery_time                          197421 non-null  str    
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  str    
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                197428 non-nu

In [202]:
df.duplicated().sum()

np.int64(0)

In [203]:
df = df.drop_duplicates()

In [204]:
df.isna().sum()

market_id                                         987
created_at                                          0
actual_delivery_time                                7
store_id                                            0
store_primary_category                           4760
order_protocol                                    995
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift_dashers                           16262
total_busy_dashers                              16262
total_outstanding_orders                        16262
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      526
total_deliveryTime                                  7
total_available_dashers                         16262
estimated_time_duration     

In [205]:
df.dropna(subset=['market_id','store_primary_category','order_protocol','estimated_store_to_consumer_driving_duration','total_deliveryTime'], inplace=True)


In [206]:
# df.dropna(subset=['total_onshift_dashers','total_busy_dashers','total_outstanding_orders','total_available_dashers'], inplace=True)


In [207]:
df["total_onshift_dashers"] = df["total_onshift_dashers"].fillna(
    df["total_onshift_dashers"].mean()
)
df["total_busy_dashers"] = df["total_busy_dashers"].fillna(
    df["total_busy_dashers"].mean()
)
df["total_outstanding_orders"] = df["total_outstanding_orders"].fillna(
    df["total_outstanding_orders"].mean()
)
df["total_available_dashers"] = df["total_available_dashers"].fillna(
    df["total_available_dashers"].mean()
)



In [208]:
df.isna().sum()

market_id                                       0
created_at                                      0
actual_delivery_time                            0
store_id                                        0
store_primary_category                          0
order_protocol                                  0
total_items                                     0
subtotal                                        0
num_distinct_items                              0
min_item_price                                  0
max_item_price                                  0
total_onshift_dashers                           0
total_busy_dashers                              0
total_outstanding_orders                        0
estimated_order_place_duration                  0
estimated_store_to_consumer_driving_duration    0
total_deliveryTime                              0
total_available_dashers                         0
estimated_time_duration                         0
dtype: int64

In [209]:
df.info()

<class 'pandas.DataFrame'>
Index: 191407 entries, 0 to 197427
Data columns (total 19 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     191407 non-null  float64
 1   created_at                                    191407 non-null  str    
 2   actual_delivery_time                          191407 non-null  str    
 3   store_id                                      191407 non-null  int64  
 4   store_primary_category                        191407 non-null  str    
 5   order_protocol                                191407 non-null  float64
 6   total_items                                   191407 non-null  int64  
 7   subtotal                                      191407 non-null  int64  
 8   num_distinct_items                            191407 non-null  int64  
 9   min_item_price                                191407 non-null  i

In [210]:
con = list(df.columns[(df.dtypes=='int64') | (df.dtypes=='float64')])
con


['market_id',
 'store_id',
 'order_protocol',
 'total_items',
 'subtotal',
 'num_distinct_items',
 'min_item_price',
 'max_item_price',
 'total_onshift_dashers',
 'total_busy_dashers',
 'total_outstanding_orders',
 'estimated_order_place_duration',
 'estimated_store_to_consumer_driving_duration',
 'total_deliveryTime',
 'total_available_dashers',
 'estimated_time_duration']

In [211]:
import numpy as np

corr_matrix = df[con].corr()
corr_matrix = corr_matrix.mask(np.eye(len(corr_matrix), dtype=bool))

corr_matrix.style.map(
    lambda x: 'background-color: red' if abs(x) > 0.8 else ''
)



,market_id,store_id,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,total_deliveryTime,total_available_dashers,estimated_time_duration
market_id,nan,0.020166,-0.020602,-0.004932,-0.015630,0.004832,-0.003561,-0.007894,0.062201,0.054718,0.057122,-0.057419,0.008067,0.005519,0.033983,-0.014359
store_id,0.020166,nan,0.015154,-0.008538,-0.012474,-0.012276,-0.009432,-0.018110,-0.022653,-0.023226,-0.020284,0.023921,0.001042,-0.004817,-0.003094,0.010093
order_protocol,-0.020602,0.015154,nan,0.009974,-0.052672,-0.021552,-0.046186,-0.091392,0.142331,0.146766,0.132167,-0.675909,-0.008811,-0.012733,0.017103,-0.265916
total_items,-0.004932,-0.008538,0.009974,nan,0.555445,0.762339,-0.393201,-0.057767,0.030808,0.027924,0.033430,-0.024936,0.003692,0.018872,0.014518,-0.006060
subtotal,-0.015630,-0.012474,-0.052672,0.555445,nan,0.680701,0.036823,0.506981,0.126179,0.121286,0.125450,0.037761,0.034608,0.044874,0.039991,0.046710
num_distinct_items,0.004832,-0.012276,-0.021552,0.762339,0.680701,nan,-0.449288,0.042865,0.062925,0.057871,0.064778,0.001875,0.021942,0.029442,0.027300,0.021202
min_item_price,-0.003561,-0.009432,-0.046186,-0.393201,0.036823,-0.449288,nan,0.543794,0.040837,0.042422,0.039710,0.053424,0.004173,0.009191,0.004028,0.024264
max_item_price,-0.007894,-0.018110,-0.091392,-0.057767,0.506981,0.042865,0.543794,nan,0.129030,0.127148,0.126694,0.084597,0.027782,0.032520,0.032112,0.058192
total_onshift_dashers,0.062201,-0.022653,0.142331,0.030808,0.126179,0.062925,0.040837,0.129030,nan,0.943725,0.936121,-0.180877,0.043243,0.130254,0.366171,-0.028583
total_busy_dashers,0.054718,-0.023226,0.146766,0.027924,0.121286,0.057871,0.042422,0.127148,0.943725,nan,0.932826,-0.186713,0.041980,0.128643,0.037802,-0.031986


## Step 3: Separate X and Y features


In [212]:
X = df[['total_outstanding_orders','total_onshift_dashers','estimated_time_duration','total_available_dashers']]
Y = df[['total_deliveryTime']]

In [213]:
X.head()

,total_outstanding_orders,total_onshift_dashers,estimated_time_duration,total_available_dashers
0,21.0,33.0,1307.0,19.0
1,2.0,1.0,1136.0,-1.0
8,18.0,8.0,735.0,2.0
14,8.0,5.0,1241.0,-1.0
15,7.0,5.0,651.0,0.0


In [214]:
Y.head()

,total_deliveryTime
0,3779.0
1,4024.0
8,1586.0
14,2273.0
15,2988.0


In [215]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [216]:
num_pipe = make_pipeline(
    # feature scaling
    StandardScaler()
).set_output(transform='pandas')

In [217]:

num_pipe

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('standardscaler', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"copy copy: bool, default=TrueIf False, try to avoid a copy and do inplace scaling instead.This is not guaranteed to always work inplace; e.g. if the data isnot a NumPy array or scipy.sparse CSR matrix, a copy may still bereturned.",True
,"with_mean with_mean: bool, default=TrueIf True, center the data before scaling.This does not work (and will raise an exception) when attempted onsparse matrices, because centering them entails building a densematrix which in common use cases is likely to be too large to fit inmemory.",True
,"with_std with_std: bool, default=TrueIf True, scale the data to unit variance (or equivalently,unit standard deviation).",True


In [218]:

X_pre = num_pipe.fit_transform(X)
X_pre.head()

,total_outstanding_orders,total_onshift_dashers,estimated_time_duration,total_available_dashers
0,-0.736762,-0.360035,1.931921,1.455099
1,-1.112761,-1.326681,1.204254,-0.370310
8,-0.796130,-1.115227,-0.502146,-0.096499
14,-0.994024,-1.205850,1.651067,-0.370310
15,-1.013814,-1.205850,-0.859596,-0.279040


## Data splitting train and test

In [219]:
# Random data splitting
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X_pre,Y,train_size=0.8,test_size=0.2,random_state=21) # reproducibility

In [220]:

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(xtrain,ytrain)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [221]:

# r2 score for training data
model.score(xtrain,ytrain)

0.019051474390710532

## Build the model

In [222]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X,Y)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [223]:
# r2 score for training data
model.score(X,Y)

0.018871326933591948

In [224]:
model.intercept_

array([-2154.79021321])

In [225]:
model.coef_

array([[3.68997925e+01, 5.29693762e+00, 7.93346276e-03, 2.17794505e+00]])

In [226]:
# Calculate the predictions
ypreds = model.predict(X)
ypreds

array([[-1153.34563713],
       [-2068.8592219 ],
       [-1438.03146173],
       ...,
       [ -468.27543335],
       [-1668.32936557],
       [-1195.5548245 ]], shape=(191407, 1))

In [227]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
mse = mean_squared_error(Y,ypreds)
mae = mean_absolute_error(Y,ypreds)
rmse = mse**(1/2)
r2 =r2_score(Y,ypreds)

In [228]:
# print the evaluation metrics
print(f"MSE : {mse}")
print(f"MAE : {mae}")
print(f"RMSE : {rmse}")
print(f"R2-score : {r2*100: .2f}%")

MSE : 215306910.3768134
MAE : 5314.597130277498
RMSE : 14673.340123394313
R2-score :  1.89%
